In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2 
from PIL import Image
import kagglehub
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

c:\Users\naqsh\.conda\envs\dl_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:

# Download latest version
path = kagglehub.dataset_download("meowmeowmeowmeowmeow/gtsrb-german-traffic-sign")


print("Path to dataset files:", path)

100%|██████████| 612M/612M [00:59<00:00, 10.8MB/s] 

Extracting files...


Path to dataset files: C:\Users\naqsh\.cache\kagglehub\datasets\meowmeowmeowmeowmeow\gtsrb-german-traffic-sign\versions\1


In [52]:
data = ImageDataGenerator(rescale=1/255.0, validation_split=0.25)
train= data.flow_from_directory(
    path+ "/Train",
    target_size = (64,64),
    batch_size= 32,
    class_mode = "categorical",
    subset="training"
    )
val= data.flow_from_directory(
    path+ "/Train",
    target_size = (64,64),
    batch_size= 32,
    class_mode = "categorical",
    subset = "validation",
    shuffle = False
)

Found 29416 images belonging to 43 classes.
Found 9793 images belonging to 43 classes.


In [ ]:
test_csv_path = os.path.join(path, 'Test.csv')

if os.path.exists(test_csv_path):
    test_df = pd.read_csv(test_csv_path)
    
    test_df['ClassId'] = test_df['ClassId'].astype(str)
    
    
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=path, 
        x_col="Path",  
        y_col="ClassId",
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical',
        shuffle=False
    )
else:
    print("Test.csv not found. Cannot load Test data via dataframe.")

Found 12630 validated image filenames belonging to 43 classes.


In [54]:
model =models.Sequential([
    layers.Input(shape=(64,64,3)),
    
    layers.Conv2D(32,(3,3), padding = 'same'),
    layers.Activation('elu'),
    layers.MaxPooling2D(2,2),
    Dropout(0.2),
    
    layers.Conv2D(64,(3,3), padding = 'same'),
    layers.Activation('elu'),
    layers.MaxPooling2D(2,2),
    Dropout(0.2),


    layers.Conv2D(128,(3,3), padding = 'same'),
    layers.Activation('elu'),
    layers.MaxPooling2D(2,2),
    Dropout(0.2),

    
    layers.Flatten(),
    layers.Dense(256),
    layers.Activation('elu'),
    layers.Dense(43, activation='softmax')  
])
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_22 (Conv2D)              │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_29 (Activation)      │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_30 (Activation)      │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_31 (Activation)      │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 256)            │     2,097,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_32 (Activation)      │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 43)             │        11,051 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,201,707 (8.40 MB)

 Trainable params: 2,201,707 (8.40 MB)

 Non-trainable params: 0 (0.00 B)

In [55]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [56]:

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5,          
    patience=3,          
    verbose=1,
    min_lr=1e-6
)

In [57]:
trained_model = model.fit(
    train,
    validation_data = val,
    epochs = 20,
    callbacks = [early_stop,lr_scheduler]
)

Epoch 1/20
920/920 ━━━━━━━━━━━━━━━━━━━━ 182s 192ms/step - accuracy: 0.8308 - loss: 0.6224 - val_accuracy: 0.8802 - val_loss: 0.5013 - learning_rate: 0.0010
Epoch 2/20
920/920 ━━━━━━━━━━━━━━━━━━━━ 156s 169ms/step - accuracy: 0.9714 - loss: 0.1033 - val_accuracy: 0.8906 - val_loss: 0.5267 - learning_rate: 0.0010
Epoch 3/20
920/920 ━━━━━━━━━━━━━━━━━━━━ 155s 168ms/step - accuracy: 0.9765 - loss: 0.0861 - val_accuracy: 0.8901 - val_loss: 0.6590 - learning_rate: 0.0010
Epoch 4/20
920/920 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - accuracy: 0.9820 - loss: 0.0738
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
920/920 ━━━━━━━━━━━━━━━━━━━━ 1329s 1s/step - accuracy: 0.9778 - loss: 0.0992 - val_accuracy: 0.8957 - val_loss: 0.7662 - learning_rate: 0.0010


In [ ]:
# Check if the class mappings match
mapping_match = train.class_indices == test_generator.class_indices

if mapping_match:
    print(" Class mappings align perfectly.")
else:
    print(" WARNING: Class mappings do not match!")
    print("Train mapping:", train.class_indices)
    print("Test mapping:", test_generator.class_indices)

✅ Class mappings align perfectly.


In [60]:
val_preds = model.predict(test_generator)

val_pred_classes = np.argmax(val_preds, axis=1)

true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())


print(classification_report(true_classes, val_pred_classes, target_names=class_labels))

395/395 ━━━━━━━━━━━━━━━━━━━━ 44s 111ms/step
              precision    recall  f1-score   support

           0       1.00      0.47      0.64        60
           1       0.88      0.94      0.91       720
          10       0.88      0.98      0.93       660
          11       0.88      0.92      0.90       420
          12       0.98      0.94      0.96       690
          13       0.99      0.97      0.98       720
          14       1.00      0.94      0.97       270
          15       0.90      1.00      0.95       210
          16       1.00      0.88      0.94       150
          17       0.95      0.93      0.94       360
          18       0.90      0.89      0.89       390
          19       0.82      0.97      0.89        60
           2       0.96      0.89      0.92       750
          20       0.77      0.96      0.85        90
          21       0.97      0.67      0.79        90
          22       0.84      0.96      0.89       120
          23       0.98      0.53    

In [66]:
model.save('traffic_classifier.h5')
print("Model saved successfully.")

Model saved successfully.
